In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as sklp
ohe=sklp.OneHotEncoder()


## 1

In [2]:
df=pd.read_csv("dataframe_exercicio_modulo_7.csv")
df.head()

,idade,tempo_educacao,estado_civil,cor,sexo,horas_por_semana,salario_anual,nome,dívida
0,39.0,13,Never-married,White,Male,40,<=50K,NaN,NaN
1,50.0,13,Married-civ-spouse,White,Male,13,<=50K,NaN,NaN
2,38.0,9,Divorced,White,Male,40,<=50K,NaN,NaN
3,53.0,7,Married-civ-spouse,Black,Male,40,<=50K,NaN,NaN
4,37.0,14,Married-civ-spouse,White,Female,40,<=50K,NaN,NaN


## 2

In [ ]:
print(df.shape)

### Temos 29170 linhas e 9 colunas.

## 3

In [ ]:
print(df.dtypes)

## 4

In [ ]:
print((df.isnull().sum()/df.shape[0]).round(3))

### Temos as variáveis nome e dívida com 100% dos dados faltantes e idade com 0.3% dos dados missing.

## 5

In [ ]:
print("Cor \n \n",df.cor.value_counts(),"\n")
print("Sexo \n \n",df.sexo.value_counts(),"\n")
print("Estado civil \n \n",df.estado_civil.value_counts(),"\n")

### È possível analisar que a variável sexo possui alguns dados faltantes que não são computados na função "isnull"

## 6

In [ ]:
dict_sexo={"?":"male","Male":"male","Female":"female"}

In [ ]:
df.sexo=df.sexo.map(dict_sexo)
df.drop(["nome","dívida"],axis=1,inplace=True)
media_idade=df.idade.mean()
df.idade=df.idade.fillna(media_idade)


In [ ]:
df.head()

In [ ]:
df.isnull().sum()

#### Adicionamos a moda na variável sexo nos valores missing "?", dropamos as variaveis nome e dívida e substituímos os valores missing da idade pela sua média

## 7

In [ ]:
mm_scaler=sklp.MinMaxScaler()
ss_scaler=sklp.StandardScaler()

In [ ]:
mm_tempo=df.tempo_educacao.values.reshape(-1,1)
ss_horas=df.horas_por_semana.values.reshape(-1,1)

### MinMax

In [ ]:
mm_scaler.fit(mm_tempo)

In [ ]:
mm_scaler.transform(mm_tempo).round(3)

### SS

In [ ]:
ss_scaler.fit(ss_horas)

In [ ]:
ss_scaler.transform(ss_horas).round(4)

In [ ]:
df1=pd.DataFrame(np.c_[mm_scaler.transform(mm_tempo),ss_scaler.transform(ss_horas)],columns=["MinMax_tempo","Standard_horas"])
df1.head()

## 8

In [ ]:
ohe_estciv=(df.estado_civil.values.reshape(-1,1))
ohe_cor=df.cor.values.reshape(-1,1)
ohe_sexo=df.sexo.values.reshape(-1,1)


In [ ]:
ohe.fit(ohe_estciv)


In [ ]:
ohe.transform(ohe_estciv).toarray()

In [ ]:
ohe.fit(ohe_sexo)

In [ ]:
ohe.transform(ohe_sexo).toarray()

In [ ]:
ohe.fit(ohe_cor)

In [ ]:
ohe.transform(ohe_cor).toarray()

## 9

In [ ]:
def detecta_out(x,lim=1.5):
    q=np.percentile(x,75)-np.percentile(x,25)
    lim_sup=np.minimum(np.percentile(x,75)+lim*q,np.max(x))
    lim_inf=np.maximum(np.percentile(x,25)-lim*q,np.min(x))
    return np.where((x<lim_inf)|(x>lim_sup),1,0)

In [ ]:
detecta_out(df.idade)

In [ ]:
df["out_idade"]=detecta_out(df.idade)
df.head()

In [ ]:
df2=df[df.out_idade==0]

In [ ]:
df2.out_idade.value_counts()

## 10

In [ ]:
def preprocessamento(df,num_faltante,cat_faltante,col_ohe,col_mmscaler,col_ssscaler,col_outlier):
    import pandas as pd
    import numpy as np
    import sklearn.preprocessing as sklp
    
    dff=df.copy()
    temp=(df.isnull().sum()/df.shape[0])>=0.8
    var_faltantes=temp.loc[temp==True].index.tolist()
    for v in var_faltantes:
        dff=dff.drop(v,axis=1)
    
    num_faltante=list(set(num_faltante).intersection(set(dff.columns.tolist())))
    cat_faltante=list(set(cat_faltante).intersection(set(dff.columns.tolist())))
    col_ohe=list(set(col_ohe).intersection(set(dff.columns.tolist())))
    col_mmscaler=list(set(col_mmscaler).intersection(set(dff.columns.tolist())))
    col_ssscaler=list(set(col_ssscaler).intersection(set(dff.columns.tolist())))
    col_outlier=list(set(col_outlier).intersection(set(dff.columns.tolist())))
    
    for v in num_faltante:
        dff[v]=dff[v].fillna(dff[v].mean())
    for v in cat_faltante:
        moda=dff[v].value_counts().idxmax()
        dff[v]=dff[v].fillna(moda)
    for v in col_ohe:
        ohe=sklp.OneHotEncoder()
        x=dff[v].values.reshape(-1,1)
        ohe.fit(x)
        dff1=pd.DataFrame(ohe.transform(x).toarray(),columns=list(ohe.categories_[0]))
        
    for v in col_mmscaler:
        mm_scaler=sklp.MinMaxScaler()
        y=dff[v].values.reshape(-1,1)
        mm_scaler.fit(y)
        dff2=pd.DataFrame(data=mm_scaler.transform(y),columns=["MinMax_tempo"])
    for v in col_ssscaler:
        ss_scaler=sklp.StandardScaler()
        z=dff[v].values.reshape(-1,1)
        ss_scaler.fit(z)
        dff3=pd.DataFrame(ss_scaler.transform(z),columns=["norm_padrao_horas"])
    for v in col_outlier:
        lim=1.5
        q=np.percentile(dff[v],75)-np.percentile(dff[v],25)
        lim_sup=np.minimum(np.percentile(dff[v],75)+lim*q,np.max(dff[v]))
        lim_inf=np.maximum(np.percentile(dff[v],25)-lim*q,np.min(dff[v]))
        dff4= pd.DataFrame(np.c_[np.where((dff[v]<lim_inf)|(dff[v]>lim_sup),1,0)],columns=["out_idade1"])
    dff=pd.concat([dff,dff1,dff2,dff3,dff4],axis=1)
    return dff    

In [ ]:
dff=preprocessamento(df=df,num_faltante=["idade"],cat_faltante=["sexo"],col_ohe=["cor"],col_mmscaler=["tempo_educacao"],col_ssscaler=["horas_por_semana"],col_outlier=["idade"])

In [ ]:
dff.columns

In [ ]:
dff.head()